In [1]:
"""Trabajo Mate 4.0.

Programa que simula una red neuronal
Fecha: 07/01/2024
Autor: Fabrizio Torrico
Correo: torricofabrizio27@gmail.com.
"""

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

Vamos a hacer un programa que simule las redes neuronales
de librerias como tensorflow o keras.

Primero vamos a definir las funciones de activacion que vamos a usar:

In [2]:
def sigmoid(x: float) -> float:
    """Función sigmoide.

    Args:
        x (float): Valor de la función.

    Returns:
        float: Valor de la función sigmoide.
    """
    return 1 / (1 + np.exp(-x))

def sigmoid_loss(y: float, y_hat: float) -> float:
    """Función de pérdida de la función sigmoide.

    Args:
        y (float): Valor real.
        y_hat (float): Valor predicho.

    Returns:
        float: Valor de la función de pérdida.
    """
    return -y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat)


Luego vamos a definir la clase de la red neuronal, que va a tener los siguientes metodos:
- `__init__(self, layers)`: Inicializa la red neuronal con las capas que se le pasan como argumento.
- `forward(self, x)`: Realiza el forward pass de la red neuronal.
- `update(self, lr)`: Actualiza los pesos de la red neuronal.

In [3]:
class NeuralLayer:
    """Clase Capa Neuronal."""

    def __init__(self, units: int):
        """Constructor de la clase Capa Neuronal.

        Args:
            units (int): Número de neuronas.
        """
        self.weights = np.random.randn(units)
        self.bias = np.random.randn()

    def feedforward(self, inputs: np.ndarray) -> np.ndarray:
        """Función de activación.

        Args:
            inputs (np.ndarray): Entradas de la capa neuronal.

        Returns:
            np.ndarray: Salida de la capa neuronal.
        """
        return np.array([neuron.feedforward(inputs) for neuron in self.neurons])

    def update(self, weights: np.ndarray, bias: float):
        """Actualizar los pesos y el bias de la capa neuronal.

        Args:
            weights (np.ndarray): Pesos de la capa neuronal.
            bias (float): Bias de la capa neuronal.
        """
        for neuron in self.neurons:
            neuron.update(weights, bias)

    def shape(self) -> tuple:
        """Obtener la forma de la capa neuronal.

        Returns:
            tuple: Forma de la capa neuronal.
        """
        return len(self.neurons), len(self.neurons[0].weights)

class NeuralNetwork:
    """Clase Red Neuronal."""

    def __init__(self, layers: list):
        """Constructor de la clase Red Neuronal.

        Args:
            layers (list): Capas de la red neuronal.
        """
        self.layers = layers

    def predict(self, inputs: np.ndarray) -> np.ndarray:
        """Predicción de la red neuronal.

        Args:
            inputs (np.ndarray): Entradas de la red neuronal.

        Returns:
            np.ndarray: Salida de la red neuronal.
        """
        for layer in self.layers:
            inputs = layer.feedforward(inputs)
        return inputs
    
    def compute_gradients(self, inputs: np.ndarray, targets: np.ndarray) -> list:
        """Cálculo de los gradientes de la red neuronal.

        Args:
            inputs (np.ndarray): Entradas de la red neuronal.
            targets (np.ndarray): Salidas esperadas de la red neuronal.

        Returns:
            list: Gradientes de la red neuronal.
        """
        gradients = []
        for layer in self.layers:
            m, n = layer.shape()
            gradients.append(
                np.zeros((m, n + 1)) 
            )


In [4]:
class NeuralNetwork:
    """Clase que representa una red neuronal."""
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        """Constructor de la clase.

        Args:
            input_size (int): Tamaño de la capa de entrada.
            hidden_size (int): Tamaño de la capa oculta.
            output_size (int): Tamaño de la capa de salida.
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.W1 = np.random.randn(self.input_size, self.hidden_size)
        self.b1 = np.zeros((1, self.hidden_size))

        self.W2 = np.random.randn(self.hidden_size, self.output_size)
        self.b2 = np.zeros((1, self.output_size))

    def forward(self, X: np.ndarray) -> np.ndarray:
        """Propagación hacia adelante.

        Args:
            X (np.ndarray): Datos de entrada.

        Returns:
            np.ndarray: Predicciones de la red.
        """
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)

        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)

        return self.a2
    
    def update(self, X: np.ndarray, y: np.ndarray, learning_rate: float) -> None:
        """Actualización de los pesos.

        Args:
            X (np.ndarray): Datos de entrada.
            y (np.ndarray): Etiquetas.
            learning_rate (float): Tasa de aprendizaje.
        """
        m = X.shape[0]

        dz2 = self.a2 - y
        dW2 = (1 / m) * np.dot(self.a1.T, dz2)
        db2 = (1 / m) * np.sum(dz2, axis=0, keepdims=True)

        dz1 = np.dot(dz2, self.W2.T) * self.a1 * (1 - self.a1)
        dW1 = (1 / m) * np.dot(X.T, dz1)
        db1 = (1 / m) * np.sum(dz1, axis=0, keepdims=True)

        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1

        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2


In [5]:
def load_data():
    x = np.load("data/X.npy")
    y = np.load("data/y.npy")
    x = x[0:1000]
    y = y[0:1000]
    return x, y

def load_weights():
    w1 = np.load("data/w1.npy")
    b1 = np.load("data/b1.npy")
    w2 = np.load("data/w2.npy")
    b2 = np.load("data/b2.npy")
    return w1, b1, w2, b2